In [2]:
!jupyter nbconvert eesardocs.ipynb --to slides --post serve 

[NbConvertApp] Converting notebook eesardocs.ipynb to slides
[NbConvertApp] Writing 264336 bytes to eesardocs.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/eesardocs.slides.html
Use Control-C to stop this server
Neues Fenster in aktueller Browsersitzung erstellt.
^C

Interrupted


In [1]:
import warnings
# these are innocuous but irritating
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# Change Detection with Sentinel-1 PolSAR imagery on the GEE

### Mort Canty
    mort.canty@gmail.com
### Joshua Rutkowski, Irmgard Niemeyer 
    Jülich Forschungszentrum, Germany
### Allan A. Nielsen, Knut Conradsen, Henning Skriver
    Technical University of Denmark
### September 2018

## Software Installation

 Pull and/or run the container with
 
     docker run -d -p 443:8888 --name=eesar mort/eesardocker 
     
 or, if you are on a Raspberry Pi,
 
     docker run -d -p 443:8888 --name=eesar mort/rpi-eesardocker

Point your browser to http://localhost:443 to see the Jupyter notebook home page. 

Open the Notebook 

    interface.ipynb 

Stop the container with

    docker stop eesar 
     
Re-start with

    docker start eesar 

### The GEE Sentinel-1 Archive

https://explorer.earthengine.google.com/#detail/COPERNICUS%2FS1_GRD

## Background

### Vector and matrix representations

A fully polarimetric SAR measures a
$2\times 2$ _scattering matrix_ $S$  at each resolution cell on the ground.
The scattering matrix relates the incident and the backscattered
electric fields $E^i$ and $E^b$ according to

$$
\pmatrix{E_h^b \cr E_v^b}
=\pmatrix{S_{hh} & S_{hv}\cr S_{vh} & S_{vv}}\pmatrix{E_h^i \cr E_v^i}.
$$

The per-pixel polarimetric information in the scattering matrix $S$, under the assumption
of reciprocity ($S_{hv} = S_{vh}$), can then be expressed as a three-component complex vector

$$
s = \pmatrix{S_{hh}\cr \sqrt{2}S_{hv}\cr S_{vv}},
$$

The total intensity is referred to as the _span_ and is the complex inner product of the vector $s$,

$$
{\rm span} = s^\top s = |S_{hh}|^2 + 2|S_{hv}|^2 + |S_{vv}|^2.
$$

The polarimetric signal is  can also be represented by taking the complex outer product of $s$ with itself:

$$
C = s s^\top = \pmatrix{ |S_{hh}|^2 & \sqrt{2}S_{hh}S_{hv}^* & S_{hh}S_{vv}^* \cr
                                     \sqrt{2}S_{hv}S_{hh}^* & 2|S_{hv}|^2 & \sqrt{2}S_{hv}S_{vv}^* \cr
                                     S_{vv}S_{hh}^* & \sqrt{2}S_{vv}S_{hv}^* & |S_{vv}|^2 }.
$$


### Multi-looking

The matrix $C$ can be averaged over the number of looks (number of adjacent cells used to average out the effect of speckle) to give an estimate of the __covariance matrix__ of each multi-look pixel:

$$
\bar{C}  ={1\over m}\sum_{\nu=1}^m  s(\nu) s(\nu)^\top = \langle  s s^\top \rangle
 = \pmatrix{ \langle |S_{hh}|^2\rangle & \langle\sqrt{2}S_{hh}S_{hv}^*\rangle & \langle S_{hh}S_{vv}^*\rangle \cr
\langle\sqrt{2} S_{hv}S_{hh}^*\rangle & \langle 2|S_{hv}|^2\rangle & \langle\sqrt{2}S_{hv}S_{vv}^*\rangle \cr
\langle S_{vv}S_{hh}^*\rangle & \langle\sqrt{2}S_{vv}S_{hv}^*\rangle & \langle |S_{vv}|^2\rangle },
$$

### Dual polarimetric imagery

The Sentinel-1 sensors operate in reduced, power-saving polarization modes, emitting only one polarization and receiving two (dual polarization) or one (single polarization). 

For vertical transmission and horizontal and vertical reception,

$$
\bar{C} = \pmatrix{ \langle |S_{vv}|^2\rangle & \langle S_{vv}S_{vh}^*\rangle \cr
\langle S_{vh}S_{vv}^*\rangle & \langle |S_{vh}|^2\rangle },
$$

The GEE archives only the diagonal (intensity) matrix elements, so we work in fact with

$$
\bar{C} = \pmatrix{ \langle |S_{vv}|^2\rangle & 0 \cr
0  & \langle |S_{vh}|^2\rangle },
$$


### Change detection, bitemporal imagery

The probability distribution of $\bar C$ is completely determined by the parameter $\Sigma$ (the covariance matrix) and by the __equivalent number of looks__ ENL. 

Given two measurements of polarized backscatter, one can set up an hypothesis test in order to decide whether or not a change has occurred. 

 $$H_0: \Sigma_1 = \Sigma_2$$ 
 
 i.e., the two observations were sampled from the same distribution and no change has occurred
 
 $$H_1: \Sigma_1\ne\Sigma_2$$
 
 in other words, there was a change.
 
Since the distributions are known, a test statistic can be formulated which allows one to decide to a desired degree of significance whether or not to reject the null hypothesis. 

### Change detection, multitemporal imagery

In the case of $k > 2$ observations this procedure can be generalized to test a null hypothesis that all of the $k$ pixels are characterized by the same $\Sigma$, against the alternative that at least one of the $\Sigma_i$, $i=1\dots k$, are different, i.e., that at least one change has taken place. 


Furthermore this so-called  __omnibus test procedure__ can be factored into a sequence of  tests involving hypotheses of the form:

$\Sigma_1 = \Sigma_2$ against $\Sigma_1 \ne \Sigma_2$,

$\Sigma_1 = \Sigma_2 = \Sigma_3$ against $\Sigma_1 = \Sigma_2 \ne \Sigma_3$,

and so forth. 

Denoting the test statistics $R^\ell_j,\ \ell = 1\dots k-1,\ j=\ell+1\dots k$, for a series of, say, $k=5$ images, we have the following tests to consider

$$
\matrix{
\ell/j &2 &3 &4 &5\cr
1 & R^1_2 & R^1_3 & R^1_4 & R^1_5 \cr
2 &       & R^2_3 & R^2_4 & R^2_5 \cr
3 &       &       & R^3_4 & R^3_5 \cr
4 &       &       &       & R^4_5 }
$$      

## The GEE interface

The interface is programmed against the GEE Python API and uses Jupyter widgets to generate the desired Sentinel-1 times series for processing.

Results (changes maps) can be previewed on-the-fly and then exported to the GEE Code Editor for visualization and animation.

In [3]:
from auxil.eeSar_seq import run
run()

TWFwKGJhc2VtYXA9eyd1cmwnOiAnaHR0cHM6Ly97c30udGlsZS5vcGVuc3RyZWV0bWFwLm9yZy97en0ve3h9L3t5fS5wbmcnLCAnbWF4X3pvb20nOiAxOSwgJ2F0dHJpYnV0aW9uJzogJ01hcCDigKY=


VkJveChjaGlsZHJlbj0oVGV4dGFyZWEodmFsdWU9dSdBbGdvcml0aG0gb3V0cHV0Jywgcm93cz0zKSwgSEJveChjaGlsZHJlbj0oVGV4dCh2YWx1ZT11JzIwMTctMDQtMDEnLCBkZXNjcmlwdGnigKY=
